# Тест логрег модели + понимание где она ошибается

## Прогон кода и предсказания

In [27]:
from pandas import read_csv, concat
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import DictVectorizer
from scipy.sparse import hstack
from sklearn.metrics import roc_auc_score
from datetime import datetime
from stop_words import get_stop_words
from tqdm.notebook import tqdm
from multiprocessing import cpu_count
from json import load as json_load
from numpy import int64, array, logspace, isnan, isfinite, nan_to_num, empty, zeros
from joblib import Parallel, delayed
from tabulate import tabulate
from pickle import load as pickle_load
from typing import List, Union
from sklearn.base import RegressorMixin, TransformerMixin
from transliterate import translit

In [4]:
def pickle_model_loader(path: str) -> Union[RegressorMixin, TransformerMixin]:
    """Load your model from pickle. Might be insecure."""
    with open(path, 'rb') as model_file:
        return pickle_load(model_file)


def safe_json_loader(path: str) -> dict:
    """Load your dict from JSON."""
    with open(path, 'rb') as model_file:
        return json_load(model_file)

In [9]:
path_to_models = "../lib/logreg_models/"

tf_idf = pickle_model_loader(path_to_models + "text_transformer.pickle")
dict_vectorizer = pickle_model_loader(path_to_models + "cat_transformer.pickle")
logreg = pickle_model_loader(path_to_models + "logreg.pickle")
regexp_dict = safe_json_loader(path_to_models + "regexps/regexp.json")

logreg.n_jobs = cpu_count()

In [10]:
categories = ['subcategory', 'category', 'region', 'city']

In [11]:
def regexp_transformer(series, regexps: dict):
    """Find the presence of regexps in your Series."""
    columns = empty((len(regexps), len(series)), dtype=int)
    for index, (_, regexp) in enumerate(regexps.items()):
        columns[index] = series.str.contains(regexp).astype(int).values
    return columns.T

In [30]:
%%time
dataframe = read_csv("../data/val.csv")
description = dataframe.description.replace(r'[\W_]+', ' ', regex=True).str.lower()

tf_idf_result = tf_idf.transform(description)
dict_vectorizer_result = dict_vectorizer.transform(dataframe[categories].to_dict('records'))
regexp_result = regexp_transformer(description, regexp_dict)

features = hstack([tf_idf_result, dict_vectorizer_result, regexp_result], format='csr')

/home/integrant/.local/lib/python3.8/site-packages/pandas/core/strings/accessor.py:101: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


CPU times: user 8.49 s, sys: 23.8 ms, total: 8.51 s
Wall time: 8.51 s


In [13]:
def auc_printer(predictions, labels, model: RegressorMixin) -> None:
    """Print your model's AUC."""
    headers = ['Model', 'Metric', 'Value']
    table = []
    model_name = type(model).__name__
    roc_auc = roc_auc_score(labels, predictions)
    table.append([model_name, 'AUC', roc_auc])
    print(tabulate(table, headers=headers, tablefmt='orgtbl'))

In [ ]:
predictions = logreg.predict_proba(features)[:, 1]

In [ ]:
auc_printer(predictions, dataframe["is_bad"].values, logreg)

## Тут я тырил новые регулярки

In [19]:
dataframe[(dataframe["is_bad"] == 1) & (dataframe["category"] == "Для бизнеса")].description.iloc[0:2].values

array(['Продам действующий бизнес. Магазин-бар разливных напитков. Расположен в отдельно стоящем здании. Ассортимент разливных напитков более 10 видов, более 400 видов снеков и закусок. Установлено видеонаблюдение, телевизор, холодильное оборудование, кондиционер, большой кегератор. Возможно работать 21/7. Рассмотрим все предложения(доп.тел.8.9.4.1.8.9.7.1.5.5.1)',
       'Продам готовый отлаженный бизнес.Магазин-бар разливных напитков. Постоянные клиенты, база поставщиков, выгодное расположение. Более 50 видов разливных напитков. Большой кегератор. Установлена система видео наблюдения. Интернет, телевизор.Расмотрим все варианты(доп.тел.8.9.2.5.8.9.7.5.1.1.5)'],
      dtype=object)

In [82]:
dataframe["predictions"] = predictions

In [91]:
dataframe[(dataframe["predictions"] < 0.1) & (dataframe["is_bad"] == 1)].description

247      Сдам квартиру семейным людям,8969 создающие611...
318      ￼Спортивные покрытия. От 190 рублейПроизводите...
321      Продам участок 54,27 сот., земли поселений (ИЖ...
328      Холодильник-морозильник Стинол-404, двухкамерн...
430      Набор BoxMezone CyberSport – это не просто игр...
                               ...                        
15872    Имеется коррозия на крыльях!8 Все узлы, агрега...
15971    Утюг электрический IR-4463 /\n /\nВас приветст...
16105    Предлагается 2-х комнатная квартира с изолиров...
16152         На 922ходу,222продаю 2050с номерами в593ор77
16225    Минимойка Xiaomi Kärcher K1 FM Cordless/\n /\n...
Name: description, Length: 268, dtype: object

## Тут я решил обучить предикторы внутри каждой из категорий

In [15]:
dataframe["category"].unique()

array(['Транспорт', 'Для бизнеса', 'Для дома и дачи', 'Личные вещи',
       'Услуги', 'Бытовая электроника', 'Недвижимость', 'Хобби и отдых',
       'Работа', 'Животные'], dtype=object)

Собственно я их обучил, тут я тестирую пайплайн для работы с тестом и поменяю конечный файл.

In [22]:
cat_names = list((category, translit(category.lower().replace(" ", "_"), 'ru', reversed=True)) for category in dataframe["category"].unique())
logregs = {}
for category, name in cat_names:
    with open("../lib/logreg_models/{0}.pickle".format(name), 'rb') as file:
        if file:
            logregs[category] = pickle_load(file)
            file.close()

In [23]:
def category_fitter(df_train, Model, params, X_train, y_train):
    models = {}
    for category in df_train["category"].unique():
        start = timer()
        print("Now working on '{0}' category.".format(category))
        if params:
            model = Model(**params[category])
        else:
            model = Model()
        model.fit(X_train[df_train["category"] == category], y_train[df_train["category"] == category])
        models[category] = model
        print("Category {0} finished, elapsed time: {1:.1f}s.\n".format(category, timer() - start))
    return models

In [28]:
def metrics_printer(features, labels, models, dataframe, models_class):
    overall_table = []
    cats_table = []
    labels_pred = zeros(labels.shape)
    for category in dataframe["category"].unique():
        cat_name = translit(category.lower().replace(" ", "_"), 'ru', reversed=True)
        model = models[category]
        labels_pred[dataframe["category"] == category] = model.predict_proba(features[dataframe["category"] == category])[:, 1]
        rocauc_category = roc_auc_score(labels[dataframe["category"] == category], labels_pred[dataframe["category"] == category])
        cats_table.append([cat_name, rocauc_category])
    rocauc = roc_auc_score(labels, labels_pred)
    overall_table.append([models_class, rocauc])
    print("Categories table:")
    print(tabulate(cats_table, headers=["Category", "AUC"], tablefmt='orgtbl'))
    print(" _____________________________________________")
    print(".____________________________________________.>")
    print("Overall table:")
    print(tabulate(overall_table, headers=['Model', 'AUC'], tablefmt='orgtbl'))

In [31]:
metrics_printer(features, dataframe["is_bad"].values, logregs, dataframe, "logreg")

Categories table:
| Category             |      AUC |
|----------------------+----------|
| transport            | 0.981164 |
| dlja_biznesa         | 0.922197 |
| dlja_doma_i_dachi    | 0.915825 |
| lichnye_veschi       | 0.813546 |
| uslugi               | 0.848407 |
| bytovaja_elektronika | 0.929878 |
| nedvizhimost'        | 0.954078 |
| hobbi_i_otdyh        | 0.87889  |
| rabota               | 0.879812 |
| zhivotnye            | 0.866757 |
 _____________________________________________
.____________________________________________.>
Overall table:
| Model   |      AUC |
|---------+----------|
| logreg  | 0.948752 |
